In [1]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('D:\GITHUB\SpamEmailClassifier\data\CaptstoneProjectData_2025.csv')
# df = df.applymap(lambda x: x.replace('\r', '').replace('\n', '') if isinstance(x, str) else x)
df.fillna("",inplace=True)
df['row_id'] = range(len(df))
print(len(df))


<>:9: SyntaxWarning: invalid escape sequence '\G'
<>:9: SyntaxWarning: invalid escape sequence '\G'
C:\Users\yck\AppData\Local\Temp\ipykernel_10220\1139296043.py:9: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_csv('D:\GITHUB\SpamEmailClassifier\data\CaptstoneProjectData_2025.csv')
2025-05-19 21:42:44,908 - INFO - NumExpr defaulting to 12 threads.


2576


C:\Users\yck\AppData\Local\Temp\ipykernel_10220\1139296043.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("",inplace=True)


In [2]:
import re

df["http_count"] = df['Body'].str.count('http://')
df["https_count"] = df['Body'].str.count('https://')
df["is_body_empty"] = df['Body'].isna()
df["is_subject_empty"] = df['Subject'].isna()

df["paragraph_count"] = df['Body'].str.count('\n')
def extract_links(text):
    #
    if text is not None and isinstance(text,str):
        return re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
df["links"] = df['Body'].apply(extract_links)

df["subject_length"] = df['Subject'].str.len()
df["body_length"] =df['Body'].str.len()

def find_max_length_in_list(lst):
    if lst is not None and len(lst)>0:
        return max(len(item) for item in lst)
    else:
        return 0
df["max_link_length"] = df["links"].apply(find_max_length_in_list)
df["link_counts"] = df["links"].apply(lambda x:len(x) if x is not None else 0)

In [3]:
import re
import unicodedata


COMMON_PUNCTUATION = set([
    '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'
])


def detect_suspicious_non_latin(text):
    suspicious_chars = []

    for char in text:
        if char.isascii():
            if char in COMMON_PUNCTUATION or char.isalnum() or char.isspace():
                continue
            else:
                suspicious_chars.append((char, 'Unusual ASCII'))
                continue

        try:
            name = unicodedata.name(char)
            if 'LATIN' not in name:
                suspicious_chars.append((char, name))
        except ValueError:
            suspicious_chars.append((char, 'No Unicode Name'))
    return len(suspicious_chars),len(set(suspicious_chars))
# Apply the function to the 'Body' column
df[["non_latin_count","non_latin_distinct_count"]] = df['Body'].apply(lambda x: detect_suspicious_non_latin(x) if isinstance(x,str) else 0).apply(pd.Series)
# df[['Body','detect_suspicious_non_latin']].sort_values(by='detect_suspicious_non_latin',ascending=False)
# ... existing code ...

In [4]:
import emoji

df["emoji_count"] = df['Body'].apply(lambda x: emoji.emoji_count(x) if isinstance(x,str) else 0)
df["emoji_distinct_count"] = df['Body'].apply(lambda x: len(emoji.distinct_emoji_list(x)) if isinstance(x,str) else 0)
df["emoji_list"] = df['Body'].apply(lambda x: emoji.emoji_list(x) if isinstance(x,str) else 0)

# print(emoji.emoji_count(email_content))
# print(len(emoji.distinct_emoji_list(email_content)))

In [5]:
df[["non_latin_distinct_count","Body"]].sort_values(by='non_latin_distinct_count',ascending=False)

,non_latin_distinct_count,Body
2446,135,"Hello, I am Manuel Franco, you have a donation..."
868,79,Invoice Due: October 12 2022\r\nAmount: USD 67...
426,33,\r\n\r\nYoulhavecalnewiVоiϲе-Nоtеl\r\n\r\nSend...
1713,30,\r\n\r\nUVIC Ѕυррогτ™®\r\nⅮυе τо геϲеɴτ υрɡгаԁ...
312,25,Notice: This message was sent from outside the...
...,...,...
990,0,Notice: This message was sent from outside the...
989,0,Notice: This message was sent from outside the...
988,0,Notice: This message was sent from outside the...
987,0,Notice: This message was sent from outside the...


In [6]:
selected_column = [ 'http_count', 'https_count',
       'is_body_empty', 'is_subject_empty', 'paragraph_count',
       'subject_length', 'body_length', 'max_link_length', 'link_counts',
       'non_latin_count', 'non_latin_distinct_count', 'emoji_count',
       'emoji_distinct_count']

df_for_kmeans = df[selected_column]


In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0)
kmeans.fit(df_for_kmeans)


KMeans(n_clusters=10, random_state=0)

In [8]:
df['label_kmeans'] = kmeans.labels_

In [9]:
df

,Subject,Body,Unnamed: 2,Unnamed: 3,row_id,http_count,https_count,is_body_empty,is_subject_empty,paragraph_count,...,subject_length,body_length,max_link_length,link_counts,non_latin_count,non_latin_distinct_count,emoji_count,emoji_distinct_count,emoji_list,label_kmeans
0,®Review your shipment details / Shipment Notif...,Notice: This message was sent from outside the...,,,0,0,2,False,False,21,...,126,962,82,2,0,0,0,0,[],5
1,Υоur ассоunt іѕ оn hоld,\r\nVotre réponse a bien été prise en compte.\...,,,1,1,2,False,False,24,...,23,1337,413,3,332,22,1,1,"[{'match_start': 1072, 'match_end': 1073, 'emo...",4
2,Completed: Invoice # KZ89TYS2564 from-Bestbuy....,Notice: This message was sent from outside the...,,,2,0,9,False,False,54,...,65,3264,503,9,2,1,2,1,"[{'match_start': 750, 'match_end': 751, 'emoji...",8
3,UVic IMPORTANT NOTICE!,Your UVIC account has been filed under the lis...,,,3,0,1,False,False,11,...,22,545,36,1,0,0,0,0,[],1
4,You have (6) Suspended incoming messages,\r\n\r\nMessage generated from uvic.ca source...,,,4,0,2,False,False,26,...,41,1284,124,1,0,0,0,0,[],4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,Update needed (5) inbox messages Pending befor...,\r\n\r\n\r\n\r\n\r\n\r\nlawfa@uvic.ca\r\n\r\nP...,,,2571,1,0,False,False,21,...,63,497,71,1,5,5,2,2,"[{'match_start': 118, 'match_end': 120, 'emoji...",1
2572,Update needed (5) inbox messages Pending befor...,\r\n\r\n\r\n\r\n\r\n\r\nlawfa@uvic.ca\r\n\r\nP...,,,2572,1,0,False,False,21,...,63,488,62,1,5,5,2,2,"[{'match_start': 118, 'match_end': 120, 'emoji...",1
2573,"Re: All Staff/Employee,","\r\nAll Staff/Employee,\r\n\r\n\r\n\r\nAs COVI...",,,2573,0,1,False,False,22,...,23,526,33,1,57,1,0,0,[],1
2574,Notification Update,Notice: This message was sent from outside the...,,,2574,0,1,False,False,23,...,19,570,50,1,0,0,0,0,[],1


In [16]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("wcukierski/enron-email-dataset", download_dir="D:\GITHUB\Sp\amEmailClassifier\data")

print("Path to dataset files:", path)


<>:4: SyntaxWarning: invalid escape sequence '\G'
<>:4: SyntaxWarning: invalid escape sequence '\G'
C:\Users\yck\AppData\Local\Temp\ipykernel_10220\3372529099.py:4: SyntaxWarning: invalid escape sequence '\G'
  path = kagglehub.dataset_download("wcukierski/enron-email-dataset", download_dir="D:\GITHUB\SpamEmailClassifier\data")
C:\Users\yck\AppData\Local\Temp\ipykernel_10220\3372529099.py:4: SyntaxWarning: invalid escape sequence '\G'
  path = kagglehub.dataset_download("wcukierski/enron-email-dataset", download_dir="D:\GITHUB\SpamEmailClassifier\data")


TypeError: dataset_download() got an unexpected keyword argument 'download_dir'

In [15]:
enron_pd = pd.read_csv(path)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\yck\\.cache\\kagglehub\\datasets\\wcukierski\\enron-email-dataset\\versions\\2'